In [4]:
import pandas as pd
import numpy as np
import glob
import tensorflow.compat.v1 as tf
# import keras_metrics
tf.disable_v2_behavior()
from matplotlib import pyplot as plt
from sklearn import metrics
from tensorflow.python.data import Dataset
import datetime
pd.options.display.max_columns = 1200
pd.options.display.max_rows = 1200
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [7]:
### Use a particular month:

# df = pd.read_csv('../data/autoperf_complete/autoperf-2019-01.csv', header=0).sample(frac=1)
df = pd.read_csv('../data/composed/mira_all.csv')

### OR use the full year

# li = []
# for filename in glob.glob("/home/luckierdodge/repos/jupyter-notebooks/data/autoperf-201*.csv"):
#     frame = pd.read_csv(filename, header=0)
#     li.append(frame)
# df = pd.concat(li, axis=0, ignore_index=True)

###

KeyboardInterrupt: 

In [6]:
### Filtering and cleaning the data before processing ###
# df.dropna(inplace=True);

###

counts = df.zero_execName.value_counts().to_dict()
df_filtered = pd.DataFrame()
for name in counts:
    if counts[name] >= 1000:
        df_filtered = df_filtered.append(df[df["zero_execName"] == name])

###

# df_filtered = df

###
df_filtered = df_filtered.sample(frac=1)

NameError: name 'df' is not defined

In [ ]:
### Use this cell to explore columns ###

# for col in df_filtered.columns:
#     if 'time' in col or 'Time' in col:
#         print(col)


# df_filtered.zero_numRanks.describe()
# df_filtered.zero_execName.value_counts().to_dict()

In [ ]:
### Use a particular subset of features:

selected_features = df_filtered[[
#     "zero_stdMpiTime",
#     "gflops",
#     "gops",
#     "ram_store_bpop",
#     "ram_fetch_bpop",
#     "USED_CORE_HOURS",
    "REQUESTED_CORE_HOURS",
#     "min_stdMpiTime",
#     "max_stdMpiTime",
#     "av_stdMpiTime",
    "zero_numRanks",
    "av_numProcessesOnNode",
#     "zero_elapsedTime",
#     "zero_numRanks",
#     "zero_elapsedCycles",
#     "zero_MPI_Barrier_callCount",
]]
features = selected_features.copy()

### OR use all of the features

# features = df_filtered.drop(['zero_execName', 'run_date', 'zero_userName', 'zero_threadMode', 'min_userName', 'min_execName', 'min_threadMode', 'max_execName', 'max_userName', 'max_threadMode', 'av_execName', 'av_userName', 'av_threadMode'], 
#     axis=1).select_dtypes(include=[np.number]).copy()

features = ((features - features.mean()) / (features.max() - features.min())).dropna(axis=1, how='any')

###

# Factorize the targets for softmax classification (trying to predict executable name)
targets = pd.DataFrame()
targets['number'] = pd.factorize(df_filtered['zero_execName'], na_sentinel=-1)[0]
# targets['number'] = pd.factorize(df_filtered['zero_userName'], na_sentinel=-1)[0]

In [ ]:
# features.describe()

In [ ]:
# Turn the pandas dataframe into tf.Keras friendly numpy arrays
def create_arrays(dataframe):
#     array = []
#     for col in dataframe.columns:
#         array.append(dataframe[col].to_numpy())
#     return array
    return dataframe.values

In [ ]:
# Separate our data into training and test sets

train_size = .8

train_examples = create_arrays(features.head(int(len(df_filtered.index) * train_size)))
train_targets = create_arrays(targets.head(int(len(df_filtered.index) * train_size)))
test_examples = create_arrays(features.tail(int(len(df_filtered.index) * (1-train_size))))
test_targets = create_arrays(targets.tail(int(len(df_filtered.index) * (1-train_size))))

In [ ]:
# Create a TF.Keras model, taking into account the number of unique executables we're targetting in our softmax layer

### More complex model to tackle a full year

# model = tf.keras.models.Sequential([
#   tf.keras.layers.Dense(2048, activation='relu', input_shape=(1,)),
#   tf.keras.layers.Dropout(.2),
#   tf.keras.layers.Dense(1024, activation='relu'),
#   tf.keras.layers.Dropout(.2),
#   tf.keras.layers.Dense(targets.number.unique().size, activation='softmax')
# ])

### Single month, simplistic neural network

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(256, activation='relu', input_shape=(len(features.columns.values),)),
  tf.keras.layers.Dense(128, activation='relu', input_shape=(1,)),
  tf.keras.layers.Dense(128, activation='relu', input_shape=(1,)),
  tf.keras.layers.Dense(128, activation='relu', input_shape=(1,)),
  tf.keras.layers.Dense(targets.number.unique().size, activation='softmax')
])

###

In [ ]:
def keras_precision(y_true, y_pred):
    fp = tf.keras.metrics.FalsePositives()
    fp.update_state(y_true, y_pred)
    false_positive = fp.result().eval()
    tp = tf.keras.metrics.TruePositives()
    tp.update_state(y_true, y_pred)
    true_positive = tp.result().eval()
    return true_positive / (true_positive + false_positive)

In [ ]:
model.compile(optimizer=tf.train.AdagradOptimizer(0.005),
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'],
            )

In [ ]:
# Tensorboard Shenanigans
log_dir = '/home/luckierdodge/repos/jupyter-notebooks/log/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, 
                                                     histogram_freq=1,
                                                     write_graph=True,
#                                                      embeddings_freq=1,
                                                     update_freq='epoch',
                                                     write_images=True)

# Train model
model.fit(train_examples, 
          train_targets, 
          epochs=10, 
#           validation_data=(test_examples, test_targets), 
          callbacks=[tensorboard_callback]
         )

In [3]:
test_acc = model.evaluate(test_examples, test_targets)
print('Test accuracy:', test_acc)

NameError: name 'model' is not defined

In [ ]:
# %reload_ext tensorboard

In [ ]:
# %tensorboard --logdir /home/luckierdodge/repos/jupyter-notebooks/log/fit